# HW2

In [1]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import scipy.stats as stats
from sklearn.ensemble import RandomForestClassifier as rfc
import random
from pathlib import Path
import sys
import matplotlib as mpl
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_confusion_matrix, roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import hinge_loss
from sklearn.ensemble import RandomForestClassifier as rfc


## Question 1

importing CSV to DataFrame

In [2]:
file = Path.cwd().joinpath('HW2_data.csv')
Diab = pd.read_csv(file)

### Of the 565 patients, we want to remove patients with at least 3 NaN.
#### Patients with 2 nan values or less: we will have the NaNs replaced with random values from a feature values (column).
#### Since this is a small number of patients from the group examined, if a bias is created it is very small, and yet the given data is large enough.


In [6]:
def nan2value_random(df):
    df = df.dropna(axis=0, thresh=15)
    for col in df:
        bank_for_col = df.loc[col].dropna()
        bank_for_col = np.random.choice(bank_for_col, size=len(df.loc[col]))
        df[col] = df.loc[col].fillna(pd.Series(bank_for_col))


    return df
clean_Diab = nan2value_random(Diab) 

KeyError: 'Age'

## Question 2

### Perform a test-train split of 20% test.

In [4]:
X = clean_Diab[['Age','Gender','Increased Urination','Increased Thirst','Sudden Weight Loss','Weakness','Increased Hunger','Genital Thrush','Visual Blurring','Itching','Irritability','Delayed Healing','Partial Paresis','Muscle Stiffness','Hair Loss','Obesity','Family History']]
y= clean_Diab[['Diagnosis']]
X_train, x_test, Y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 10, stratify=y)


## Question 3

## 3a: 

In [5]:
##create binary train, test DataFrames (except for age feature):
X_train_binary = X_train.replace(['Yes','Female','Positive'],value = 1)
X_train_binary = X_train_binary.replace(['No','Male','Negative'],value = 0)
x_test_binary = x_test.replace(['Yes','Female','Positive'],value = 1)
x_test_binary = x_test_binary.replace(['No','Male','Negative'],value = 0)
#create a dictionary with features and values as %train, %test, %delta
list_train = [None]*16
list_test = [None]*16
delta = [None]*16
features_dictionary={}
features_dictionary['Positive Feature']=['%Train', '%Test', '%Delta']
for i in range(0,16):
    list_train[i] = X_train_binary.iloc[:,i+1].sum()*(100/len(X_train_binary))
    list_test[i] = x_test_binary.iloc[:,i+1].sum()*(100/len(x_test_binary))
    delta[i] = abs(list_train[i]-list_test[i])
    features_dictionary[clean_Diab.columns[i+1]]= [list_train[i], list_test[i], delta[i]]

df_features_dictionary = pd.DataFrame.from_dict(features_dictionary).T
print(df_features_dictionary)

                           0        1         2
Positive Feature      %Train    %Test    %Delta
Gender               37.4165  31.8584   5.55807
Increased Urination  48.7751  47.7876  0.987445
Increased Thirst     43.8753  44.2478  0.372509
Sudden Weight Loss   40.5345  44.2478   3.71327
Weakness             57.0156   59.292   2.27645
Increased Hunger     45.2116  44.2478  0.963794
Genital Thrush       19.5991  30.9735   11.3743
Visual Blurring      45.2116  43.3628   1.84875
Itching              47.8842  49.5575   1.67334
Irritability          23.608  23.8938  0.285787
Delayed Healing      46.7706  45.1327   1.63786
Partial Paresis      43.6526   39.823   3.82955
Muscle Stiffness     36.7483  35.3982    1.3501
Hair Loss            35.6347  36.2832  0.648442
Obesity              17.5947  14.1593   3.43536
Diagnosis            48.9978  55.7522   6.75444


## 3b:

In [ ]:
#create a dictionary that shows the relationship between feature and label.
#keys: 1.positive feature 2.negative feature
#values for each key: 1. number of positive diagnosis 2.number of negative diagnosis
#We create a bar plot after defining a dictionary for a feature
label_feature_relationship={}
label_feature_relationship['Female'] = {}
label_feature_relationship['Female']['Positive'] = len(clean_Diab[(clean_Diab.Gender.str.contains('Female')) & (clean_Diab.Diagnosis.str.contains('Positive'))])
label_feature_relationship['Female']['Negative'] = len(clean_Diab[(clean_Diab.Gender.str.contains('Female')) & (clean_Diab.Diagnosis.str.contains('Negative'))])

label_feature_relationship['Male'] = {}
label_feature_relationship['Male']['Positive'] = len(clean_Diab[(clean_Diab.Gender.str.contains('Male')) & (clean_Diab.Diagnosis.str.contains('Positive'))])
label_feature_relationship['Male']['Negative'] = len(clean_Diab[(clean_Diab.Gender.str.contains('Male')) & (clean_Diab.Diagnosis.str.contains('Negative'))])

df = pd.DataFrame.from_dict(label_feature_relationship)
df = df.T
df.plot.bar(rot=0, title='Gender')
plt.ylabel('Counts')
plt.show()

for i in range(2,16):
    title = clean_Diab.columns[i]
    label_feature_relationship = {}
    label_feature_relationship["Has %s" %title] = {}
    label_feature_relationship["Has %s" %title]['Positive'] = len(clean_Diab[(clean_Diab[title].str.contains('Yes')) & clean_Diab.Diagnosis.str.contains('Positive')])
    label_feature_relationship["Has %s" %title]['Negative'] = len(clean_Diab[(clean_Diab[title].str.contains('Yes')) & clean_Diab.Diagnosis.str.contains('Negative')])
    label_feature_relationship["No %s" % title] = {}
    label_feature_relationship["No %s" % title]['Positive'] = len(clean_Diab[(clean_Diab[title].str.contains('No')) & clean_Diab.Diagnosis.str.contains('Positive')])
    label_feature_relationship["No %s" % title]['Negative'] = len(clean_Diab[(clean_Diab[title].str.contains('No')) & clean_Diab.Diagnosis.str.contains('Negative')])
    df = pd.DataFrame.from_dict(label_feature_relationship)
    df = df.T
    df.plot.bar(rot=0, title=title)
    plt.ylabel('Counts')
    plt.show()

label_feature_relationship = {}
label_feature_relationship["Has Family History"] = {}
label_feature_relationship["Has Family History"]['Positive'] = len(clean_Diab.loc[clean_Diab['Family History']==1 & clean_Diab.Diagnosis.str.contains('Positive')])
label_feature_relationship["Has Family History"]['Negative'] = len(clean_Diab.loc[clean_Diab['Family History']==1 & clean_Diab.Diagnosis.str.contains('Negative')])
label_feature_relationship["No Family History"] = {}
label_feature_relationship["No Family History"]['Positive'] = len(clean_Diab.loc[clean_Diab['Family History']==0 & clean_Diab.Diagnosis.str.contains('Positive')])
label_feature_relationship["No Family History"]['Negative'] = len(clean_Diab.loc[clean_Diab['Family History']==0 & clean_Diab.Diagnosis.str.contains('Negative')])
df = pd.DataFrame.from_dict(label_feature_relationship)
df = df.T
df.plot.bar(rot=0, title='Family History')
plt.ylabel('Counts')
plt.show()

## 3c:

In [ ]:
#1. Is there an age from which the chance of getting sick increases significantly?- A histogram is drawn showing a connection between age and a positive diagnosis:
ax = clean_Diab.hist(column='Age', bins= 100, figsize=(12,8), color='#86bf91', zorder=2, rwidth=0.9)
ax = ax[0]
for x in ax:
    x.set_ylabel("Count", labelpad=20, weight='bold', size=12)

In [ ]:
#2. Compare age histogram for positive diagnosis and age histogram for negative diagnosis:
positive_age_series = clean_Diab[clean_Diab.Diagnosis.str.contains('Positive')]['Age']
negative_age_series = clean_Diab[clean_Diab.Diagnosis.str.contains('Negative')]['Age']
plt.hist(positive_age_series, bins=100, label='Positive Diagnosis')
plt.hist(negative_age_series, bins=100, label='Negative Diagnosis')
plt.xlabel('Age')
plt.ylabel('Count')
plt.legend(loc='upper right')
plt.show()

In [ ]:
#3. Pie chart of positive/negative diagnosis
positive_count = 0
negative_count = 0
for idx, value in enumerate(clean_Diab['Diagnosis']):
    if value =='Positive':
        positive_count +=1
    else:
        negative_count +=1
labels = ('Positive', 'Negative')
sizes = [positive_count, negative_count]
colors = ['lightcoral', 'yellowgreen']
figureObject, axesObject = plt.subplots()
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.title("Positive and negative diagnosis percentages")
plt.show()

## Question 4

In [ ]:
#Encoding our data as one hot vectors
X_binary = X.replace(['Yes','Female','Positive'],value = 1)
X_binary = X_binary.replace(['No','Male','Negative'],value = 0)
enc = OneHotEncoder( sparse=False,handle_unknown='ignore')
onehotvector=enc.fit_transform(X_binary.drop(['Age'], axis=1))
print(onehotvector)

## Qusetion 5

In [ ]:
# K cross fold+ SVM ( linear for 'svm_kernel':['linear'], non linear for 'svm_kernel':['rbf'])
# Linear SVM model
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, random_state=10, shuffle=True)
svc = SVC(probability=True)
C = np.array([0.001, 0.01, 1, 10, 100, 1000])
pipe = Pipeline(steps=[('scale', StandardScaler()), ('svm', svc)])
svm_lin = GridSearchCV(estimator=pipe,
             param_grid={'svm_kernel':['linear'], 'svm_C':C}, scoring=['roc_auc'],  cv=skf, refit='roc_auc', verbose=3, return_train_score=True)

svm_lin.fit(X_train_binary, Y_train)
best_svm_lin = svm_lin.best_estimator_


calc_TN = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[0, 0]
calc_FP = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[0, 1]
calc_FN = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[1, 0]
calc_TP = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[1, 1]

y_pred_test_lin = best_svm_lin.predict(x_test_binary)
y_pred_proba_test_lin = best_svm_lin.predict_proba(x_test_binary)
lin_loss = hinge_loss(y_test, y_pred_proba_test_lin[:,1])
LSVM_score = roc_auc_score(y_test, y_pred_proba_test_lin[:,1])

TN = calc_TN(y_test, y_pred_test_lin)
FP = calc_FP(y_test, y_pred_test_lin)
FN = calc_FN(y_test, y_pred_test_lin)
TP = calc_TP(y_test, y_pred_test_lin)
Se = TP/(TP+FN)
Sp = TN/(TN+FP)
PPV = TP/(TP+FP)
NPV = TN/(TN+FN)
Acc = (TP+TN)/(TP+TN+FP+FN)
F1 = (2*Se*PPV)/(Se+PPV)

#Non- linear SVM (RBF Kernel):
pipe_rbf = Pipeline(steps=[('scale', StandardScaler()), ('svm', svc)])
svm_rbf = GridSearchCV(estimator=pipe,
             param_grid={'svm_kernel':['rbf'], 'svm_C':C}, scoring=['roc_auc'],  cv=skf, refit='roc_auc', verbose=3, return_train_score=True)

svm_rbf.fit(X_train_binary, Y_train)
best_svm_rbf = svm_rbf.best_estimator_

y_pred_test_rbf = best_svm_rbf.predict(x_test_binary)
y_pred_proba_test_rbf = best_svm_rbf.predict_proba(x_test_binary)
rbf_loss = hinge_loss(y_test, y_pred_proba_test_rbf[:,1])
rbf_SVM_score = roc_auc_score(y_test, y_pred_proba_test_rbf[:,1])

TN_nl = calc_TN(y_test, y_pred_test_rbf)
FP_nl = calc_FP(y_test, y_pred_test_rbf)
FN_nl = calc_FN(y_test, y_pred_test_rbf)
TP_nl = calc_TP(y_test, y_pred_test_rbf)
Se_nl = TP/(TP+FN)
Sp_nl = TN/(TN+FP)
PPV_nl = TP/(TP+FP)
NPV_nl = TN/(TN+FN)
Acc_nl = (TP+TN)/(TP+TN+FP+FN)
F1_nl = (2*Se*PPV)/(Se+PPV)

print("svm with  linear kernel:")
print(f'Sensitivity is {Se:.2f}')
print(f'Specificity is {Sp:.2f}')
print(f'PPV is {PPV:.2f}')
print(f'NPV is {NPV:.2f}')
print(f'Accuracy is {Acc:.2f}')
print(f'F1 is {F1:.2f}')
print(f'The Linear Loss is {lin_loss:.2f}')
print(f'AUC is {LSVM_score:.2f}')
print("\n svm with rbf kernel:")
print(f' Sensitivity is {Se_nl:.2f}')
print(f' Specificity is {Sp_nl:.2f}')
print(f' PPV is {PPV_nl:.2f}')
print(f' NPV is {NPV_nl:.2f}')
print(f' Accuracy is {Acc_nl:.2f}')
print(f' F1 is {F1_nl:.2f}')
print(f' Loss is {rbf_loss:.2f}')
print(f' AUC is {rbf_SVM_score:.2f}')

## Question 6- Feature Selection

In [ ]:
#Random Forest Network
clf = rfc(n_estimators=10)
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train_binary)
clf.fit(X_train_scale, Y_train)
w_ = clf.feature_importances_
features=['Age','Gender','Increased Urination','Increased Thirst','Sudden Weight Loss','Weakness','Increased Hunger','Genital Thrush','Visual Blurring','Itching','Irritability','Delayed Healing','Partial Paresis','Muscle Stiffness','Hair Loss','Obesity','Family History']
x = np.arange(len(features))

plt.bar(x, w_,0.5, color='c')
plt.xticks(x,features,rotation=90);
plt.ylabel("weights", fontsize=12)
plt.title("Feature Weights- RFC")

plt.show()
